In [1]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
from image_utils import ImageVisualizer, show_progress

In [ ]:
show_progress(rf"D:\nnUNet_results\Dataset046_Perfusion\nnUNetTrainer__nnUNetPlans__3d_fullres\fold_0\progress.png")

In [4]:
df = pd.read_csv("/mnt/d/CTH_CTP.rdvdata/DB-CTH_CTP.csv") #CSV with presorted CTA images only
cta_acc_set = set((df['Accession number']))
full_acc_set = set(os.listdir("/mnt/d/CTH"))
full_acc_set = set(os.listdir("/mnt/d/CTH"))

In [7]:
source_dir = '/mnt/d/CTH_archive/DICOM'
destination_dir = '/mnt/d/CTH_archive/CBF_COLOR/'

if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Iterate through each accession number in the set
for acc in tqdm(cta_acc_set):
    source_path = os.path.join(source_dir, str(acc))  # Path to the directory named by the accession number
    destination_path = os.path.join(destination_dir, str(acc))

    # Check if the directory exists before trying to copy
    if os.path.exists(source_path):
        # Check if the destination directory already exists to avoid 'FileExistsError'
        if not os.path.exists(destination_path):
            shutil.copytree(source_path, destination_path)
            print(f"Directory {source_path} copied successfully to {destination_path}.")
        else:
            print(f"Destination directory {destination_path} already exists.")
    else:
        print(f"Source directory {source_path} does not exist.")


100%|██████████| 134/134 [00:00<00:00, 1476.95it/s]

Source directory /mnt/d/CTH_archive/DICOM/1004678146 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1003672067 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1003151362 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1004158985 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1005006347 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1004559889 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1004359186 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1004591633 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1004728851 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1003177507 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/101091364 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1003177508 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1006162471 does not exist.
Source directory /mnt/d/CTH_archive/DICOM/1001881641 does not exist.
Source directory /mnt/d/CTH_archive

In [ ]:
import os
import pydicom
import shutil
from tqdm import tqdm

def rename_folders_with_patient_name(base_path):
    log = []  # Collect logs for troubleshooting

    for accession_dir in tqdm(next(os.walk(base_path))[1]):
        full_path = os.path.join(base_path, accession_dir)
        patient_name_found = False
        log.append(f"Processing {accession_dir}")

        for root, dirs, files in os.walk(full_path):
            for file in files:
                if file.lower().endswith('.dcm'):
                    dicom_path = os.path.join(root, file)
                    try:
                        dicom = pydicom.dcmread(dicom_path, stop_before_pixels=True)
                        patient_name = str(dicom.PatientName)
                        parts = patient_name.split('^')
                        if len(parts) > 1:
                            patient_name = '_'.join(parts)
                        patient_name = ''.join(e for e in patient_name if e.isalnum() or e == '_')
                        new_folder_path = os.path.join(base_path, patient_name)

                        if full_path != new_folder_path:
                            if not os.path.exists(new_folder_path):
                                shutil.move(full_path, new_folder_path)
                                log.append(f"Renamed {full_path} to {new_folder_path}")
                                patient_name_found = True
                                break
                            else:
                                log.append(f"Target folder {new_folder_path} already exists.")
                        else:
                            log.append(f"No need to rename {full_path}.")
                    except Exception as e:
                        log.append(f"Error processing {dicom_path}: {str(e)}")
            if patient_name_found:
                break

    for entry in log:
        print(entry)

base_path = "D:\\CTH_archive\\CBV_RENAME"
rename_folders_with_patient_name(base_path)

In [ ]:
import os
import pydicom
from collections import defaultdict
from tqdm import tqdm
def list_dicom_series_by_patient(base_path):
    # Dictionary to store series descriptions organized by patient name
    patient_series = defaultdict(list)

    # Iterate over each directory in the base path
    for patient_dir in tqdm(os.listdir(base_path)):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):  # Ensure it is a directory
            for root, dirs, files in os.walk(patient_path):
                for file in files:
                    if file.lower().endswith('.dcm'):
                        dicom_path = os.path.join(root, file)
                        try:
                            # Load DICOM file
                            dicom = pydicom.dcmread(dicom_path, stop_before_pixels=True)
                            series_description = dicom.get('SeriesDescription', 'No Description')
                            # Add series description to the list for the patient
                            patient_series[patient_dir].append(series_description)
                        except Exception as e:
                            print(f"Error processing {dicom_path}: {e}")

    # Output the results
    for patient, descriptions in patient_series.items():
        print(f"Patient: {patient}")
        unique_descriptions = set(descriptions)  # Remove duplicates
        for description in unique_descriptions:
            print(f"  Series Description: {description}")

    return patient_series


base_path = "D:\\CTH_archive\\CTA"
dicom_series_by_patient = list_dicom_series_by_patient(base_path)

# Given the dictionary `dicom_series_by_patient` and a list of terms to ignore
ignore_terms = [
    "SCAN DOC", "No Description", "Cor", "Sag", "MIP", "Dose",
    "Bayer", "Smart", "SCANNED", "offline", "Patient", "Scout", "Screen", "SUMMARY", "SCANDOC"
    "reformat", "LVO", "monitoring", "AIDOC", "LAT", "1 CE", "OLIVE", "Topo", "ISOTROPIC", "THINS"
]

# Convert the ignore terms to uppercase for case-insensitive comparison
ignore_terms = [term.upper() for term in ignore_terms]

# Iterate through the dictionary and remove descriptions containing any ignore terms
for patient in list(dicom_series_by_patient.keys()):
    # Use a set to store unique series descriptions
    unique_series = set()
    for description in dicom_series_by_patient[patient]:
        if not any(ignore_term in description.upper() for ignore_term in ignore_terms):
            unique_series.add(description)
    # Update the dictionary with unique filtered series
    dicom_series_by_patient[patient] = list(unique_series)

# Now, dicom_series_by_patient is updated with unique series descriptions
# Print the updated list to confirm
for patient, series in sorted(dicom_series_by_patient.items()):
    print(f"Patient: {patient}")
    for description in sorted(series):  # Sorting to have a consistent output order
        print(f"  Series Description: {description}")


In [1]:
from extract_utils import DICOMSeriesExtractor

desired_series = {
    "VIZ CBF COLORED", "VIZ CBF"
}

ignore_terms = [ #dont need ignore terms for this searhc
#    "SCAN DOC", "No Description", "Cor", "Sag", "MIP", "Dose", "Bayer", "Smart",
#    "SCANNED", "offline", "Patient", "Scout", "Screen", "SUMMARY", "SCANDOC", "reformat",
#    "LVO", "monitoring", "AIDOC", "LAT", "1 CE", "OLIVE", "Topo", "ISOTROPIC", "THINS"
]
extractor = DICOMSeriesExtractor(desired_series, ignore_terms)
base_path = "/mnt/d/CTH_archive/DICOM"
extractor.validate_dicom_series(base_path)


Starting validation of DICOM series...
Processing patient directory: ALFORD_BARBARA 4024996
Validated series for ALFORD_BARBARA 4024996: {'VIZ CBF COLORED', 'VIZ CBF'}
Processing patient directory: ALLAH_MAJUSTICE 2621774
Validated series for ALLAH_MAJUSTICE 2621774: {'VIZ CBF'}
Processing patient directory: BATTLE_MARIA 8399298
Validated series for BATTLE_MARIA 8399298: {'VIZ CBF'}
Processing patient directory: BAUM_ROBERT 6035108
Validated series for BAUM_ROBERT 6035108: {'VIZ CBF COLORED', 'VIZ CBF'}
Processing patient directory: BILLIPS_JAMES 6272788
Validated series for BILLIPS_JAMES 6272788: {'VIZ CBF COLORED', 'VIZ CBF'}
Processing patient directory: BOGER_DAVID_S 2532249
Validated series for BOGER_DAVID_S 2532249: {'VIZ CBF'}
Processing patient directory: BROWN_ANTHONY F341525
Validated series for BROWN_ANTHONY F341525: {'VIZ CBF'}
Processing patient directory: CAMPAGNA_HARRY_D E304749
Validated series for CAMPAGNA_HARRY_D E304749: {'VIZ CBF'}
Processing patient directory: CANI

In [10]:
# Define the target directory for the converted NIfTI files
target_base = "/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_CBF"
extractor.copy_dicom(target_base)

Starting conversion of DICOM files to NIfTI format...
Failed to convert DICOM for ALFORD_BARBARA 4024996: Exception thrown in SimpleITK ImageSeriesReader_Execute: /tmp/SimpleITK-build/ITK-prefix/include/ITK-5.3/itkImageSeriesReader.hxx:351:
ITK ERROR: ImageSeriesReader(0x4aea060): Size mismatch! The size of  /mnt/d/CTH_archive/DICOM/ALFORD_BARBARA 4024996/2019-11-30 163115/IMG-0001-00002.dcm is [888, 532, 1] and does not match the required size [888, 513, 1] from file /mnt/d/CTH_archive/DICOM/ALFORD_BARBARA 4024996/2019-11-30 163115/IMG-0001-00001.dcm
/mnt/d/CTH_archive/DICOM/ALLAH_MAJUSTICE 2621774/2020-07-21 200446/IMG-0006-00001.dcm /home/nicholasjprimiano/ML/img2img-turbo/data/CTH_CBF/ALLAH_MAJUSTICE 2621774.nii
Converted and saved DICOM from ALLAH_MAJUSTICE 2621774 to /home/nicholasjprimiano/ML/img2img-turbo/data/CTH_CBF/ALLAH_MAJUSTICE 2621774.nii
/mnt/d/CTH_archive/DICOM/BATTLE_MARIA 8399298/2021-01-03 084826/IMG-0038-00001.dcm /home/nicholasjprimiano/ML/img2img-turbo/data/CTH_C

In [ ]:
from extract_utils import DICOMSeriesExtractor
target_base = "D:\\CTH_archive\\CBV_NIFTI"

extractor.copy_dicom(target_base)

In [ ]:
import os 
len(os.listdir(target_base))

In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

def scroll_through_nifti(file_path):
    # Load the NIfTI file
    nifti = nib.load(file_path)
    data = nifti.get_fdata()

    # Define a function to plot a slice
    def plot_slice(slice_no):
        plt.figure(figsize=(8, 8))
        plt.imshow(data[:, :, slice_no], cmap='gray')
        plt.axis('off')
        plt.title(f'Slice {slice_no}')
        plt.show()

    # Create an interactive slider for slice selection
    interact(plot_slice, slice_no=IntSlider(min=0, max=data.shape[2] - 1, step=1, value=data.shape[2] // 2))

# Example usage in a Jupyter Notebook
file_path = "D:\CTH_archive\CTA_STRIPPED\SCOUFARAS_DESPINA.nii"
scroll_through_nifti(file_path)


In [ ]:
from image_utils import ImageVisualizer, show_progress
DATASET_NUM = "041"
FOLD_NUM = 0

show_progress(fr"D:\nnUNet_results\Dataset{DATASET_NUM}_Perfusion\nnUNetTrainer__nnUNetPlans__3d_fullres\fold_{FOLD_NUM}\progress.png")

In [ ]:
from image_utils import convert_series_to_nifti
convert_series_to_nifti("D:\CTH_archive\CBF", "D:\CTH_archive\CBF_NIFTI")

In [ ]:
import os

def collect_dicom_files(base_path):
    """Collect all DICOM files organized by patient name."""
    dicom_files_by_patient = {}

    for patient_dir in os.listdir(base_path):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(patient_path) for file in files if file.lower().endswith('.dcm')]
            dicom_files_by_patient[patient_dir] = dicom_files

    return dicom_files_by_patient

In [ ]:
base_path = 'D:\\CTH_archive\\DICOM'
dicom_files_by_patient = collect_dicom_files(base_path)

# Print the DICOM files for each patient
for patient, dicom_files in dicom_files_by_patient.items():
    print(f"PATIENT: {patient}")
    for file in dicom_files:
        print(file)

In [ ]:
import os
import pydicom

def validate_dicom_series(base_path):
    """Validate DICOM series across all patients in the specified directory."""
    print("Starting validation of DICOM series...")
    for patient_dir in os.listdir(base_path):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            print(f"Processing patient directory: {patient_dir}")
            all_series_descriptions = set()  # Track all series descriptions found
            dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(patient_path) for file in files if file.lower().endswith('.dcm')]
            for dicom_file in dicom_files:
                try:
                    dicom = pydicom.dcmread(dicom_file, stop_before_pixels=True)
                    series_description = getattr(dicom, 'SeriesDescription', 'Unknown').strip().upper()
                    all_series_descriptions.add(series_description)  # Collect all descriptions
                except Exception as e:
                    print(f"Error processing {dicom_file}: {e}")

            #print(f"Available series for patient '{patient_dir}': {all_series_descriptions}")  # Print all series descriptions

            if "VIZ CBF" not in map(str.upper, all_series_descriptions):
                print(f"Error: No 'VIZ CBF' series identified for patient '{patient_dir}'.")
            elif list(map(str.upper, all_series_descriptions)).count("VIZ CBF") > 1:
                print(f"Error: More than one 'VIZ CBF' series identified for patient '{patient_dir}'.")
            else:
                print(f"Validated 'VIZ CBF' series for {patient_dir}.")

    print("Validation complete.")


base_path = 'D:\\CTH_archive\\DICOM'
validate_dicom_series(base_path)

In [ ]:
import os
import pydicom
import shutil
from tqdm import tqdm

def copy_dicom(base_path, target_base):
    """Copy 'VIZ CBF' DICOM series to the specified directory."""
    print("Starting copying of DICOM files...")
    patient_dirs = os.listdir(base_path)
    for patient_dir in tqdm(patient_dirs, desc="Copying DICOM files"):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            dicom_files = [os.path.join(root, file) for root, dirs, files in os.walk(patient_path) for file in files if file.lower().endswith('.dcm')]
            for dicom_file in dicom_files:
                try:
                    dicom = pydicom.dcmread(dicom_file, stop_before_pixels=True)
                    series_description = getattr(dicom, 'SeriesDescription', 'Unknown').strip().upper()
                    if series_description == "VIZ CBF":
                        # Create a directory for the patient if it doesn't exist
                        patient_name = patient_dir.split()[0]  # Get just the patient name
                        patient_dir = os.path.join(target_base, patient_name)
                        os.makedirs(patient_dir, exist_ok=True)

                        # Copy the DICOM file to the patient's directory
                        output_file = os.path.join(patient_dir, os.path.basename(dicom_file))
                        shutil.copyfile(dicom_file, output_file)
                except Exception as e:
                    print(f"Failed to copy DICOM file {dicom_file} for patient {patient_dir}: {e}")

    print("All DICOM copying completed.")

base_path = 'D:\\CTH_archive\\DICOM'
target_base = 'D:\\CTH_archive\\CBF'
copy_dicom(base_path, target_base)

In [ ]:
import os
import SimpleITK as sitk

def convert_series_to_nifti(input_directory, output_file):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(input_directory)
    reader.SetFileNames(dicom_names)
    image_series = reader.Execute()
    
    # Convert to numpy array to manipulate the pixel data directly
    img_array = sitk.GetArrayFromImage(image_series)
        
    # Convert the numpy array back to a SimpleITK Image
    processed_image = sitk.GetImageFromArray(img_array)
    processed_image.SetSpacing(image_series.GetSpacing())
    processed_image.SetOrigin(image_series.GetOrigin())
    processed_image.SetDirection(image_series.GetDirection())

    # Write the processed image as a NIfTI file
    sitk.WriteImage(processed_image, output_file)

def convert_all_series_to_nifti(base_path, target_base):
    """Convert all DICOM series in the base directory to NIfTI format and save them in the target directory."""
    print("Starting conversion of DICOM series to NIfTI format...")
    for patient_dir in os.listdir(base_path):
        patient_path = os.path.join(base_path, patient_dir)
        if os.path.isdir(patient_path):
            output_file = os.path.join(target_base, f"{patient_dir}.nii")
            try:
                convert_series_to_nifti(patient_path, output_file)
                print(f"Converted DICOM series from {patient_dir} to {output_file}")
            except Exception as e:
                print(f"Failed to convert DICOM series for patient {patient_dir}: {e}")

    print("All conversions completed.")


base_path = 'D:\\CTH_archive\\CBF'
target_base = 'D:\\CTH_archive\\CBF_NIFTI'
convert_all_series_to_nifti(base_path, target_base)

In [ ]:
import os
import pydicom
from tqdm import tqdm
from PIL import Image
import numpy as np

def blackout_image(image_array, blackout_left=45, blackout_top=35):
    # Black out the left column and the top row
    image_array[:, :blackout_left] = 0
    image_array[:blackout_top, :] = 0
    return image_array

def convert_to_png(dicom_file, output_dir):
    try:
        dicom_data = pydicom.dcmread(dicom_file)
        if hasattr(dicom_data, 'PixelData'):
            pixel_array = dicom_data.pixel_array

            # Check if the image is RGB
            if len(pixel_array.shape) == 3 and pixel_array.shape[2] == 3:
                # Black out specified regions
                blacked_out_array = blackout_image(pixel_array)

                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                png_file = os.path.join(output_dir, os.path.basename(dicom_file).replace('.dcm', '.png'))

                # Convert to PIL image
                image = Image.fromarray(blacked_out_array.astype(np.uint8), mode='RGB')
                image.save(png_file, format='PNG')
                return png_file
            else:
                print(f"Image is not in expected RGB format: {dicom_file}")
                return None
        else:
            print(f"No PixelData in {dicom_file}")
            return None
    except Exception as e:
        print(f"Could not convert {dicom_file} to PNG: {e}")
        return None

def search_and_convert_all_dicom_series(base_dir, primary_series_desc, secondary_series_desc, output_dir):
    patient_dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]

    for patient_dir in tqdm(patient_dirs, desc="Processing patients"):
        primary_series_found = False
        secondary_series_found = False
        primary_series_files = []
        secondary_series_files = []
        file_count = sum([len(files) for r, d, files in os.walk(patient_dir) if any(file.endswith(".dcm") for file in files)])

        with tqdm(total=file_count, desc=f"Processing DICOM files in {os.path.basename(patient_dir)}") as pbar:
            for root, _, files in os.walk(patient_dir):
                for file in files:
                    if file.endswith(".dcm"):
                        try:
                            file_path = os.path.join(root, file)
                            dicom_data = pydicom.dcmread(file_path, stop_before_pixels=True)
                            series_desc = dicom_data.SeriesDescription

                            if series_desc == primary_series_desc:
                                primary_series_files.append(file_path)
                                primary_series_found = True
                            elif series_desc == secondary_series_desc and not primary_series_found:
                                secondary_series_files.append(file_path)
                                secondary_series_found = True

                            pbar.update(1)
                        except Exception as e:
                            print(f"Could not read file {file_path}: {e}")

        # Use primary series if found, otherwise use secondary series
        series_files = primary_series_files if primary_series_found else secondary_series_files

        if series_files:
            with tqdm(total=len(series_files), desc=f"Converting to PNG for {os.path.basename(patient_dir)}") as pbar:
                for file_path in series_files:
                    output_patient_dir = os.path.join(output_dir, os.path.basename(patient_dir))
                    convert_to_png(file_path, output_patient_dir)
                    pbar.update(1)

base_dir = r'/mnt/d/CTH_archive/DICOM'
output_dir = r'/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX'
primary_series_desc = "VIZ TMAX COLORED"
secondary_series_desc = "VIZ TMAX"

search_and_convert_all_dicom_series(base_dir, primary_series_desc, secondary_series_desc, output_dir)


In [ ]:
import os
import pydicom
import SimpleITK as sitk
from tqdm import tqdm

def blackout_image(image_array, blackout_left=45, blackout_top=35):
    # Black out the left column and the top row
    image_array[:, :blackout_left] = 0
    image_array[:blackout_top, :] = 0
    return image_array

def convert_to_nifti(dicom_files, output_file):
    try:
        print(f"Processing {len(dicom_files)} DICOM files for conversion.")
        # Create a dictionary to group files by their dimensions
        size_dict = {}
        for dicom_file in dicom_files:
            dicom_data = pydicom.dcmread(dicom_file)
            size = dicom_data.Rows, dicom_data.Columns
            if size not in size_dict:
                size_dict[size] = []
            size_dict[size].append(dicom_file)

        # Find the group with the most files
        max_size = max(size_dict, key=lambda x: len(size_dict[x]))
        dicom_files_consistent = size_dict[max_size]
        print(f"Selected {len(dicom_files_consistent)} consistent DICOM files based on dimension.")

        # Read the DICOM series
        reader = sitk.ImageSeriesReader()
        reader.SetFileNames(dicom_files_consistent)
        image = reader.Execute()
        print(f"Image read successfully. Dimension: {image.GetDimension()} Size: {image.GetSize()}")

        # Convert SimpleITK image to numpy array
        image_array = sitk.GetArrayFromImage(image)
        print("Applying blackout to image slices.")

        # Apply blackout to each axial slice
        for i in range(image_array.shape[0]):
            image_array[i] = blackout_image(image_array[i])

        # Convert numpy array back to SimpleITK image
        image = sitk.GetImageFromArray(image_array)
        image.SetSpacing(reader.GetOutput().GetSpacing())
        image.SetOrigin(reader.GetOutput().GetOrigin())
        image.SetDirection(reader.GetOutput().GetDirection())
        print("Blackout applied. Converting back to SimpleITK image.")

        # Save the processed image as NIfTI
        sitk.WriteImage(image, output_file)
        print(f"NIfTI file saved: {output_file}")
        return output_file

    except Exception as e:
        print(f"Error during conversion: {e}")
        return None

def search_and_convert_all_dicom_series(base_dir, primary_series_desc, secondary_series_desc, output_dir):
    dirs = os.listdir(base_dir)
    patient_dirs = [os.path.join(base_dir, d) for d in tqdm(dirs, desc="Scanning directories") if os.path.isdir(os.path.join(base_dir, d))]
    print(f"Found {len(patient_dirs)} patient directories.")

    for patient_dir in tqdm(patient_dirs, desc="Processing patients"):
        print(f"Processing directory: {patient_dir}")
        primary_series_found = False
        secondary_series_found = False
        primary_series_files = []
        secondary_series_files = []
        file_count = sum([len(files) for r, d, files in os.walk(patient_dir) if any(file.endswith(".dcm") for file in files)])
        print(f"Found {file_count} DICOM files.")

        with tqdm(total=file_count, desc=f"Processing DICOM files in {os.path.basename(patient_dir)}") as pbar:
            for root, _, files in os.walk(patient_dir):
                for file in files:
                    if file.endswith(".dcm"):
                        file_path = os.path.join(root, file)
                        try:
                            dicom_data = pydicom.dcmread(file_path, stop_before_pixels=True)
                            series_desc = dicom_data.SeriesDescription

                            if series_desc == primary_series_desc:
                                primary_series_files.append(file_path)
                                primary_series_found = True
                            elif series_desc == secondary_series_desc and not primary_series_found:
                                secondary_series_files.append(file_path)
                                secondary_series_found = True
                        except Exception as e:
                            print(f"Error reading file {file_path}: {e}")
                        finally:
                            pbar.update(1)

        # Use primary series if found, otherwise use secondary series
        series_files = primary_series_files if primary_series_found else secondary_series_files
        print(f"Selected {len(series_files)} files from {'primary' if primary_series_found else 'secondary'} series.")

        if series_files:
            output_patient_dir = os.path.join(output_dir, os.path.basename(patient_dir))
            convert_to_nifti(series_files, output_patient_dir + '.nii')

base_dir = r'/mnt/d/CTH_archive/DICOM'
output_dir = r'/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_NIFTI'
primary_series_desc = "VIZ TMAX COLORED"
secondary_series_desc = "VIZ TMAX"

search_and_convert_all_dicom_series(base_dir, primary_series_desc, secondary_series_desc, output_dir)


In [ ]:
import os
import nibabel as nib
import imageio
from tqdm import tqdm
import numpy as np

# Define the source and target directories
src_dir = '/mnt/d/CTH_archive/CTH_STRIPPED'
tgt_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/CTH'

# Ensure the target directory exists
os.makedirs(tgt_dir, exist_ok=True)

# Clipping range in Hounsfield units
hu_min = -50
hu_max = 100
threshold_value = 0  # This is a placeholder, adjust based on your data characteristics

# Process each .nii file in the source directory
for file_name in tqdm(os.listdir(src_dir)):
    if file_name.endswith('.nii') or file_name.endswith('.nii.gz'):
        # Load the .nii file
        file_path = os.path.join(src_dir, file_name)
        nii_img = nib.load(file_path)
        img_data = nii_img.get_fdata()

        # Apply a fixed threshold to create a brain mask
        brain_mask = img_data > threshold_value

        # Clip the image data to the range [-50, 100] Hounsfield units
        img_data = np.clip(img_data, hu_min, hu_max)

        # Normalize the image data to the range [0, 65535] for 16-bit
        normalized_data = (img_data - hu_min) / (hu_max - hu_min)
        normalized_data = (normalized_data * 65535).astype(np.uint16)

        # Apply the brain mask
        normalized_data[~brain_mask] = 0

        # Save each axial slice as a 16-bit .png if it's not completely black
        for i in range(normalized_data.shape[2]):
            slice_data = normalized_data[:, :, i]
            if np.any(slice_data):  # Check if the slice is not completely black
                slice_file_name = f'{os.path.splitext(file_name)[0]}_slice_{i:04d}.png'
                slice_file_path = os.path.join(tgt_dir, slice_file_name)
                imageio.imwrite(slice_file_path, slice_data)

print('Conversion complete.')


In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm
import random

# Define the source and target directories
src_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX'
train_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/train_A'
test_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/test_A'

# Ensure the target directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Collect all .png file paths
all_files = []
for root, _, files in os.walk(src_dir):
    for file in files:
        if file.endswith('.png'):
            all_files.append(os.path.join(root, file))

# Shuffle the list of files to ensure random selection for testing
random.shuffle(all_files)

# Split the files: 90% for training and 10% for testing
split_index = int(0.9 * len(all_files))
train_files = all_files[:split_index]
test_files = all_files[split_index:]

# Function to copy files and keep track of their new paths
def copy_files(file_list, target_dir, start_index):
    file_mapping = []
    for i, file_path in enumerate(tqdm(file_list)):
        new_file_name = f'{start_index + i:06d}.png'
        new_file_path = os.path.join(target_dir, new_file_name)
        shutil.copy(file_path, new_file_path)
        file_mapping.append((file_path, new_file_path))
    return file_mapping

# Copy training files
train_mapping = copy_files(train_files, train_dir, 1)

# Copy testing files
test_mapping = copy_files(test_files, test_dir, 1)

# Create a DataFrame to keep track of the conversion
columns = ['Original Path', 'New Path']
train_df = pd.DataFrame(train_mapping, columns=columns)
test_df = pd.DataFrame(test_mapping, columns=columns)

# Save the DataFrame to a CSV file
csv_path = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/file_conversion_mapping.csv'
combined_df = pd.concat([train_df, test_df], keys=['Train', 'Test'])
combined_df.to_csv(csv_path, index=False)

print('File conversion and splitting complete. CSV file saved.')


In [ ]:
import os
import shutil
import pandas as pd
from tqdm import tqdm
import random

# Define the source and target directories
src_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/CTH'
train_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/CTH/train_B'
test_dir = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/CTH/test_B'

# Ensure the target directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Collect all .png file paths
all_files = [os.path.join(src_dir, f) for f in os.listdir(src_dir) if f.endswith('.png')]

# Shuffle the list of files to ensure random selection for testing
random.shuffle(all_files)

# Split the files: 90% for training and 10% for testing
split_index = int(0.9 * len(all_files))
train_files = all_files[:split_index]
test_files = all_files[split_index:]

# Function to copy files and keep track of their new paths
def copy_files(file_list, target_dir, start_index):
    file_mapping = []
    for i, file_path in enumerate(tqdm(file_list)):
        new_file_name = f'{start_index + i:06d}.png'
        new_file_path = os.path.join(target_dir, new_file_name)
        shutil.copy(file_path, new_file_path)
        file_mapping.append((file_path, new_file_path))
    return file_mapping

# Copy training files
train_mapping = copy_files(train_files, train_dir, 1)

# Copy testing files
test_mapping = copy_files(test_files, test_dir, 1)

# Create a DataFrame to keep track of the conversion
columns = ['Original Path', 'New Path']
train_df = pd.DataFrame(train_mapping, columns=columns)
test_df = pd.DataFrame(test_mapping, columns=columns)

# Save the DataFrame to a CSV file
csv_path = '/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/CTH/file_conversion_mapping.csv'
combined_df = pd.concat([train_df, test_df], keys=['Train', 'Test'])
combined_df.to_csv(csv_path, index=False)

print('File conversion and splitting complete. CSV file saved.')


In [ ]:
import os
import nibabel as nib
import numpy as np
from tqdm import tqdm

# Directory containing the NIFTI files
nifti_dir = "/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_NIFTI_REG"

# Function to flip the image along the anterior-posterior axis
def flip_anterior_posterior(nifti_file):
    img = nib.load(nifti_file)
    data = img.get_fdata()
    # Flip the data along the second axis (anterior-posterior)
    flipped_data = np.flip(data, axis=1)
    flipped_img = nib.Nifti1Image(flipped_data, img.affine, img.header)
    nib.save(flipped_img, nifti_file)

# Process each NIFTI file in the directory
for filename in tqdm(os.listdir(nifti_dir)):
    if filename.endswith(".nii"):
        file_path = os.path.join(nifti_dir, filename)
        flip_anterior_posterior(file_path)


In [ ]:
import nibabel as nib
import numpy as np

pt_name = "ALFORD_BARBARA"
# Load the NIFTI file
file_path = f"/home/nicholasjprimiano/ML/img2img-turbo/data/CTH_TMAX/TMAX_NIFTI_REG/{pt_name}.nii"
nifti_img = nib.load(file_path)

# Get the image data
data = nifti_img.get_fdata()

# Flip the data along the anterior-posterior axis (typically the second axis for NIFTI files)
flipped_data = np.flip(data, axis=1)

# Create a new NIFTI image with the flipped data
flipped_img = nib.Nifti1Image(flipped_data, nifti_img.affine, nifti_img.header)

# Save the flipped image
save_path = file_path.replace(f"{pt_name}.nii", f"{pt_name}_flipped.nii")
flipped_img.to_filename(save_path)
